# 🗄️ Teste de Banco de Dados

Notebook para testar conexão com PostgreSQL do projeto AI Data Engineering.

**Configuração:**
- Porta: 5433 (evitar conflito com PostgreSQL local)
- Database: ai_data_engineering
- Usuário: postgres
- Localização: Blumenau, SC

In [2]:
# Importações
import sys
import os
import psycopg2
from datetime import datetime

# Adicionar src ao path
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

print("📦 Módulos importados com sucesso!")
print(f"🐍 Python: {sys.version}")
print(f"🕐 Teste realizado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📦 Módulos importados com sucesso!
🐍 Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
🕐 Teste realizado em: 2025-09-28 23:55:02


In [3]:
# Configurações de conexão
DB_CONFIG = {
    'host': 'localhost',
    'port': 5433,
    'database': 'ai_data_engineering',
    'user': 'postgres',
    'password': 'postgres123'
}

print("🔧 Configurações de banco:")
for key, value in DB_CONFIG.items():
    if key == 'password':
        print(f"   {key}: {'*' * len(value)}")
    else:
        print(f"   {key}: {value}")
print()

🔧 Configurações de banco:
   host: localhost
   port: 5433
   database: ai_data_engineering
   user: postgres
   password: ***********



In [5]:
# Teste de conexão básica
print("🔌 Testando conexão com PostgreSQL...")
print("-" * 40)

try:
    conn = psycopg2.connect(**DB_CONFIG)
    print("✅ Conexão estabelecida com sucesso!")
    
    # Criar cursor
    cursor = conn.cursor()
    
    # Teste básico
    cursor.execute("SELECT version();")
    version = cursor.fetchone()[0]
    print(f"📊 PostgreSQL: {version[:50]}...")
    
    # Verificar database
    cursor.execute("SELECT current_database();")
    db_name = cursor.fetchone()[0]
    print(f"🗄️ Database: {db_name}")
    
    # Verificar usuário
    cursor.execute("SELECT current_user;")
    user = cursor.fetchone()[0]
    print(f"👤 Usuário: {user}")
    
    # Verificar encoding
    cursor.execute("SELECT current_setting('client_encoding');")
    encoding = cursor.fetchone()[0]
    print(f"🔤 Encoding: {encoding}")
    
    # Verificar timezone
    cursor.execute("SELECT current_setting('timezone);")
    timezone = cursor.fetchone()[0]
    print(f"🌍 Timezone: {timezone}")
    
    # Teste com dados em português
    cursor.execute("SELECT 'Blumenau' as cidade, 'Santa Catarina' as estado")
    portuguese_data = cursor.fetchone()
    print(f"📍 Dados em português: {portuguese_data[0]} - {portuguese_data[1]}")
    
    # Fechar conexão
    cursor.close()
    conn.close()
    
    print("\n🎉 CONEXÃO FUNCIONANDO PERFEITAMENTE!")
    
except Exception as e:
    print(f"❌ Erro na conexão: {e}")
    print(f"🔍 Tipo do erro: {type(e).__name__}")
    
    print("\n💡 Soluções possíveis:")
    print("1. Verificar se Docker PostgreSQL está rodando:")
    print("   docker-compose up -d postgres")
    print("\n2. Verificar se porta 5433 está correta no docker-compose.yml")
    print("\n3. Verificar se não há conflito de portas")

🔌 Testando conexão com PostgreSQL...
----------------------------------------
❌ Erro na conexão: connection to server at "localhost" (::1), port 5433 failed: FATAL:  password authentication failed for user "postgres"

🔍 Tipo do erro: OperationalError

💡 Soluções possíveis:
1. Verificar se Docker PostgreSQL está rodando:
   docker-compose up -d postgres

2. Verificar se porta 5433 está correta no docker-compose.yml

3. Verificar se não há conflito de portas


In [ ]:
# Teste com SQLAlchemy (se conexão anterior funcionou)
try:
    from sqlalchemy import create_engine, text
    
    print("🔌 Testando SQLAlchemy...")
    
    # URL de conexão
    database_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    
    # Criar engine
    engine = create_engine(database_url, echo=False)
    
    # Teste de conexão
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1 as teste"))
        test_value = result.fetchone()[0]
        
        print(f"✅ SQLAlchemy funcionando! Valor de teste: {test_value}")
        
        # Teste adicional
        result = connection.execute(text("SELECT NOW() as agora"))
        now = result.fetchone()[0]
        print(f"🕐 Hora atual do banco: {now}")
        
except Exception as e:
    print(f"❌ Erro com SQLAlchemy: {e}")
except ImportError:
    print("❌ SQLAlchemy não instalado")
except NameError:
    print("❌ Execute primeiro a célula de conexão básica")

In [ ]:
# Verificar tabelas existentes
try:
    from sqlalchemy import create_engine, text
    
    print("📋 Verificando tabelas do banco...")
    
    database_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    engine = create_engine(database_url, echo=False)
    
    with engine.connect() as connection:
        # Listar tabelas
        result = connection.execute(text("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public'
            ORDER BY table_name
        """))
        
        tables = [row[0] for row in result.fetchall()]
        
        if tables:
            print(f"📊 Tabelas encontradas ({len(tables)}):")
            for table in tables:
                print(f"   - {table}")
        else:
            print("📭 Nenhuma tabela encontrada (banco vazio)")
            print("💡 Execute o ETL para criar tabelas e carregar dados")
            
except Exception as e:
    print(f"❌ Erro ao verificar tabelas: {e}")

In [ ]:
# Teste da configuração do projeto
try:
    print("🔧 Testando configuração do projeto...")
    
    from config.database import test_connection, engine
    
    # Testar função do projeto
    success = test_connection()
    
    if success:
        print("✅ Configuração do projeto funcionando!")
        
        # Teste com engine do projeto
        from sqlalchemy import text
        with engine.connect() as connection:
            result = connection.execute(text("SELECT 'AI Data Engineering' as projeto"))
            projeto = result.fetchone()[0]
            print(f"🚀 Projeto configurado: {projeto}")
            
    else:
        print("❌ Configuração do projeto falhou")
        
except Exception as e:
    print(f"❌ Erro ao testar configuração do projeto: {e}")
    print(f"\n💡 Verificar:")
    print("1. Se porta está correta em src/config/database.py")
print("2. Se senha está correta")
print("3. Se Docker está rodando corretamente")

In [ ]:
# Resumo final
print("🎯 RESUMO DO TESTE DE BANCO DE DADOS")
print("=" * 50)
print(f"🌍 Localização: Blumenau, SC - Brasil")
print(f"🗄️ Banco: PostgreSQL")
print(f"🔌 Porta: 5433 (Docker)")
print(f"📊 Database: ai_data_engineering")
print(f"👤 Usuário: postgres")
print(f"🕐 Horário: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

print("\n✅ Se todos os testes passaram:")
print("   • Conexão PostgreSQL funcionando")
print("   • Encoding UTF-8 OK")
print("   • Configuração do projeto OK")
print("\n💡 Próximos passos:")
print("   1. Criar tabelas do projeto")
print("   2. Carregar dados BMW")
print("   3. Testar consultas SQL")
print("   4. Implementar análise de dados")

print("\n🔧 Comandos úteis:")
print("   \ndocker-compose up -d postgres  # Subir banco")
print("   docker-compose down             # Parar containers")
print("   docker ps                       # Ver containers")